# Rate change per press release

In this file we combine the press conference text data with changes on ECB key rates, assigning each text a value representing the (average) change in the rates. For example if in june 2023 DFR rate was 3.25 and in july 2023 the same rate was 3.50, the june's conference had a negative effect of -0.25. Currently this is a bit counterintuitive (as the change is actually +0.25). Subject due to change. 

In [27]:
import pandas as pd

press_releases = pd.read_csv('ecb_scraper/outputs/clean_text_excl_nums.csv')
key_rates = pd.read_csv('ECB_key_rates/ECB Data Portal_20231016130234.csv')

# Merge press_releases and key_rates on date
press_releases['date'] = pd.to_datetime(press_releases['date'])
key_rates['date'] = pd.to_datetime(key_rates['DATE'])
merged = press_releases.merge(key_rates, on='date', how='left')

data = merged.drop(columns=['DATE', 'TIME PERIOD'])
data = data.rename(columns={data.columns[2]: 'DFR', data.columns[3]: 'MLF', data.columns[4]: ' MRO'})

data['DFR_change'] = data['DFR'].diff()
data['MLF_change'] = data['MLF'].diff()
data['MRO_change'] = data[' MRO'].diff()
data['AVG_change'] = data[['DFR_change', 'MLF_change', 'MRO_change']].mean(axis=1)

policy_effect = data[['date', 'text', 'AVG_change']]
policy_effect.to_csv('policy_change_rates.csv', index=False)

        date                                               text   DFR   MLF  \
0 2023-09-14  frankfurt main septemb good afternoon vicepres...  3.75  4.50   
1 2023-07-27  frankfurt main juli good afternoon vicepresid ...  3.50  4.25   
2 2023-06-15  frankfurt main june good afternoon vicepresid ...  3.25  4.00   
3 2023-05-04  frankfurt main good afternoon vicepresid welco...  3.00  3.75   
4 2023-03-16  frankfurt main march good afternoon vicepresid...  2.50  3.25   

    MRO  DFR_change  MLF_change  MRO_change  AVG_change  
0  4.25         NaN         NaN         NaN         NaN  
1  4.00       -0.25       -0.25       -0.25       -0.25  
2  3.75       -0.25       -0.25       -0.25       -0.25  
3  3.50       -0.25       -0.25       -0.25       -0.25  
4  3.00       -0.50       -0.50       -0.50       -0.50  
